# Project 2
by:
- Rebecca Kuhlman
- Michael Amberg
- Sam Yao

## Business Understanding
Identifying the type of brain tumor a patient has is an important step in figuring out the treatment plan of a patient. They can be diagnosed via MRI imaging, leading to interest in using machine learning to diagnose the patient. Having a second opinion on brain tumor diagnoses would help improve patient care and outcomes, and lessen stress on doctors.
In this dataset, there is glioma, meningioma, and pituitary tumors, as well as MRI images with no tumors.
Glioma tumors are usually malignant, while meningioma and pituitary tumors are usually benign. Different types of tumors are made of different types of cells and have a location where they are most likely to be located.
More information can be found at: https://www.mayoclinic.org/diseases-conditions/brain-tumor/symptoms-causes/syc-20350084
There are many other types of tumors that future algorithms will be need to address. The majority of other types of tumors are more common in children, while the set we are dealing with are all adult brain images.

Because the consequences of the algorithm being wrong are high, our model must be 90% or above. It should be noted that the majority of misdiagnose of brain tumors happen before a brain scan or related test is ordered.
https://paulandperkins.com/brain-tumors/


## Data Preparation

Several helpful sources that helped this part of the section include:
- [1] https://pillow.readthedocs.io/en/stable/handbook/tutorial.html
- [2] https://towardsdatascience.com/loading-custom-image-dataset-for-deep-learning-models-part-1-d64fa7aaeca6

In [68]:
import pandas as pd
import numpy as np
import os
from PIL import Image # Utilized Source [2]

img = Image.open("./Training/glioma_tumor/gg (1).jpg") # Utilized Source [1]
img_arr = np.array(img)
new_arr = list()
for x in img_arr:
    for y in x:
        new_arr.append(y)
print(len(new_arr))
# This method creates the data, wether training or testing, in the form we desire
# Uses code from source [2] to create the training datasets
def create_dataset(img_folder):
    # Read through all files in "./Training"
    img_data_array=[]
    class_name=[]
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            image= np.array(Image.open(image_path))
            image = np.resize(image, (1,262144,3)) #Vectorizes each image
            image = image.astype('float32')
            #image /= 255  
            img_data_array.append(image)
            class_name.append(dir1)
    # return array with training data.
    return img_data_array, class_name

262144


In [69]:
df_training, training_classes = create_dataset("./Training")
df_testing, testing_classes = create_dataset("./Testing")

In [73]:
df_training[0].shape

(1, 262144, 3)

## Data Reduction

## Exceptional Work 😡